In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
sc=SparkContext()
sc

<SparkContext master=yarn appName=pyspark-shell>

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
spark

## Load Year 2019 data (Total 50.4GB)

In [5]:
data2019 = spark.read.option("header", "false").csv("s3://noaa-global-hourly-pds/2019/*.csv")

In [6]:
data2019.cache()

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string, _c30: string, _c31: string, _c32: string, _c33: string, _c34: string, _c35: string, _c36: string, _c37: string, _c38: string, _c39: string, _c40: string, _c41: string, _c42: string, _c43: string, _c44: string, _c45: string, _c46: string, _c47: string, _c48: string, _c49: string, _c50: string, _c51: string, _c52: string, _c53: string, _c54: string, _c55: string, _c56: string, _c57: string, _c58: string]

In [7]:
data2019.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [8]:
data2019.show(3)

+-----------+-------------------+------+--------+---------+---------+--------------------+-----------+---------+---------------+--------------+-----------+------------+-------+-------+-------+-----------+----+----+-----------+-------------+--------+--------+--------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+------+---------+---------+---------+---------+------------------+------------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------------+-------------+-------------+----+----+----+----+----+----+----+----+-------+--------------------+----+
|        _c0|                _c1|   _c2|     _c3|      _c4|      _c5|                 _c6|        _c7|      _c8|            _c9|          _c10|       _c11|        _c12|   _c13|   _c14|   _c15|       _c16|_c17|_c18|       _c19|         _c20|    _c21|    _c22|    _c2

## Data Cleaning

In [15]:
from pyspark.sql import SparkSession
#from pyspark.sql import functions._
#import pyspark.spark.sql.{SQLContext, Row, DataFrame, Column}
from pyspark.sql import Row,DataFrame,Column
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
from pyspark.sql import functions as f
#from pyspark.ml import feature.VectorAssembler

In [10]:
import datetime

In [23]:
#data cleaning
data1 = data2019.where('_c0!= "STATION" ')
#select useful columns
data2 = data1.select(col("_c0"),col("_c1"), col("_c3"),col("_c4"),col("_c5"),
                     col("_c10"),col("_c11"),col("_c12"), col("_c13"), col("_c14"),
                     col("_c15"),col("_c16"))
data2 = data2.select(col("_c0").alias("station"), col("_c1").alias("date"), 
                     col("_c3").alias("latitude"),col("_c4").alias("longitude"),
                     col("_c5").alias("elevation"),
                     f.split(data2['_c10'], ',')[3].alias("wind"),
                     f.split(data2['_c11'], ',')[0].alias("cig"),
                     f.split(data2['_c12'], ',')[0].alias("vis"),
                     f.split(data2['_c13'], ',')[0].alias("temp"),
                     f.split(data2['_c14'], ',')[0].alias("dew"), 
                     f.split(data2['_c15'], ',')[0].alias("slp"), 
                         f.split(data2['_c16'], ',')[1].alias("liquid"),
                         f.split(data2['_c16'], ',')[0].alias("liqhr")
                        )
data2.show(5)

+-----------+-------------------+--------+---------+---------+----+-----+------+-----+-----+-----+------+-----+
|    station|               date|latitude|longitude|elevation|wind|  cig|   vis| temp|  dew|  slp|liquid|liqhr|
+-----------+-------------------+--------+---------+---------+----+-----+------+-----+-----+-----+------+-----+
|99999927516|2019-01-01T00:00:00| 71.3213| -156.611|      4.6|0064|99999|999999|-0203|+9999|99999|  0000|   01|
|99999927516|2019-01-01T00:05:00| 71.3213| -156.611|      4.6|9999|99999|999999|-0203|+9999|99999|  null| null|
|99999927516|2019-01-01T00:10:00| 71.3213| -156.611|      4.6|9999|99999|999999|-0202|+9999|99999|  null| null|
|99999927516|2019-01-01T00:15:00| 71.3213| -156.611|      4.6|9999|99999|999999|-0202|+9999|99999|  null| null|
|99999927516|2019-01-01T00:20:00| 71.3213| -156.611|      4.6|9999|99999|999999|-0201|+9999|99999|  null| null|
+-----------+-------------------+--------+---------+---------+----+-----+------+-----+-----+-----+------

In [40]:
#convert data type to float
data3=data2.withColumn("temp", data2["temp"].cast("float"))
data3=data3.withColumn("vis", data3["vis"].cast("float"))
data3=data3.withColumn("liquid", data3["liquid"].cast("float"))
data3=data3.withColumn("liqhr", data3["liqhr"].cast("float"))
data3=data3.withColumn("latitude", data3["latitude"].cast("float"))
data3=data3.withColumn("longitude", data3["longitude"].cast("float"))
data3=data3.withColumn("elevation", data3["elevation"].cast("float"))
data3=data3.withColumn("wind", data3["wind"].cast("float"))
data3=data3.withColumn("cig", data3["cig"].cast("float"))
data3=data3.withColumn("dew", data3["dew"].cast("float"))
data3=data3.withColumn("slp", data3["slp"].cast("float"))
data3.show(5)

+-----------+-------------------+--------+---------+---------+------+-------+--------+------+------+-------+------+-----+
|    station|               date|latitude|longitude|elevation|  wind|    cig|     vis|  temp|   dew|    slp|liquid|liqhr|
+-----------+-------------------+--------+---------+---------+------+-------+--------+------+------+-------+------+-----+
|99999927516|2019-01-01T00:00:00| 71.3213| -156.611|      4.6|  64.0|99999.0|999999.0|-203.0|9999.0|99999.0|   0.0|  1.0|
|99999927516|2019-01-01T00:05:00| 71.3213| -156.611|      4.6|9999.0|99999.0|999999.0|-203.0|9999.0|99999.0|  null| null|
|99999927516|2019-01-01T00:10:00| 71.3213| -156.611|      4.6|9999.0|99999.0|999999.0|-202.0|9999.0|99999.0|  null| null|
|99999927516|2019-01-01T00:15:00| 71.3213| -156.611|      4.6|9999.0|99999.0|999999.0|-202.0|9999.0|99999.0|  null| null|
|99999927516|2019-01-01T00:20:00| 71.3213| -156.611|      4.6|9999.0|99999.0|999999.0|-201.0|9999.0|99999.0|  null| null|
+-----------+-----------

In [41]:
data3.printSchema()

root
 |-- station: string (nullable = true)
 |-- date: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- elevation: float (nullable = true)
 |-- wind: float (nullable = true)
 |-- cig: float (nullable = true)
 |-- vis: float (nullable = true)
 |-- temp: float (nullable = true)
 |-- dew: float (nullable = true)
 |-- slp: float (nullable = true)
 |-- liquid: float (nullable = true)
 |-- liqhr: float (nullable = true)



In [25]:
# Drop rows with NA or missing values in pyspark
#data3=data3.dropna()

In [42]:
# drop 9999 as NA according to format guide https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf
data3 = data3.filter(data3.latitude != '99999')
data3 = data3.filter(data3.longitude != '999999')
data3 = data3.filter(data3.elevation != '9999')
data3 = data3.filter(data3.wind != '9999')
data3 = data3.filter(data3.cig != '99999')
data3 = data3.filter(data3.temp != '9999')
data3 = data3.filter(data3.vis != '999999')
data3 = data3.filter(data3.slp != '99999')
data3 = data3.filter(data3.dew != '9999')
data3 = data3.filter(data3.liquid != '9999')
data3 = data3.filter(data3.liqhr != '99')
data3.show(5)

+-----------+-------------------+--------+----------+---------+----+------+-------+------+------+-------+------+-----+
|    station|               date|latitude| longitude|elevation|wind|   cig|    vis|  temp|   dew|    slp|liquid|liqhr|
+-----------+-------------------+--------+----------+---------+----+------+-------+------+------+-------+------+-----+
|72475023176|2019-01-01T03:52:00|38.41667|-113.01667|   1536.2|41.0| 396.0| 2012.0| -83.0|-106.0|10193.0|   0.0|  1.0|
|72475023176|2019-01-01T04:52:00|38.41667|-113.01667|   1536.2|57.0| 335.0| 1609.0| -78.0| -94.0|10192.0|   5.0|  1.0|
|72475023176|2019-01-01T05:52:00|38.41667|-113.01667|   1536.2|62.0| 762.0| 4828.0| -83.0|-106.0|10196.0|   2.0|  1.0|
|72475023176|2019-01-01T06:52:00|38.41667|-113.01667|   1536.2|46.0| 610.0| 3219.0| -89.0|-111.0|10205.0|   0.0|  1.0|
|72475023176|2019-01-01T07:52:00|38.41667|-113.01667|   1536.2|67.0|1280.0|16093.0|-100.0|-133.0|10212.0|   0.0|  1.0|
+-----------+-------------------+--------+------

In [43]:
# rescale data according to format
data4=data3.withColumn("temp", data3["temp"]/10)
data4=data4.withColumn("wind", data4["wind"]/10)
data4=data4.withColumn("dew", data4["dew"]/10)
data4=data4.withColumn("slp", data4["slp"]/10)

#liquid depth per hour
data4=data4.withColumn("liquid", data4["liquid"]/10/data4["liqhr"])
data4.show(5)

+-----------+-------------------+--------+----------+---------+----+------+-------+-----+-----+------+------+-----+
|    station|               date|latitude| longitude|elevation|wind|   cig|    vis| temp|  dew|   slp|liquid|liqhr|
+-----------+-------------------+--------+----------+---------+----+------+-------+-----+-----+------+------+-----+
|72475023176|2019-01-01T03:52:00|38.41667|-113.01667|   1536.2| 4.1| 396.0| 2012.0| -8.3|-10.6|1019.3|   0.0|  1.0|
|72475023176|2019-01-01T04:52:00|38.41667|-113.01667|   1536.2| 5.7| 335.0| 1609.0| -7.8| -9.4|1019.2|   0.5|  1.0|
|72475023176|2019-01-01T05:52:00|38.41667|-113.01667|   1536.2| 6.2| 762.0| 4828.0| -8.3|-10.6|1019.6|   0.2|  1.0|
|72475023176|2019-01-01T06:52:00|38.41667|-113.01667|   1536.2| 4.6| 610.0| 3219.0| -8.9|-11.1|1020.5|   0.0|  1.0|
|72475023176|2019-01-01T07:52:00|38.41667|-113.01667|   1536.2| 6.7|1280.0|16093.0|-10.0|-13.3|1021.2|   0.0|  1.0|
+-----------+-------------------+--------+----------+---------+----+----

In [30]:
#data5 = data4.drop("liqhr") 
#data5.show(5)

+-----------+-------------------+--------+----------+---------+----+------+-------+-----+-----+------+------+
|    station|               date|latitude| longitude|elevation|wind|   cig|    vis| temp|  dew|   slp|liquid|
+-----------+-------------------+--------+----------+---------+----+------+-------+-----+-----+------+------+
|72475023176|2019-01-01T03:52:00|38.41667|-113.01667|   1536.2| 4.1| 396.0| 2012.0| -8.3|-10.6|1019.3|   0.0|
|72475023176|2019-01-01T04:52:00|38.41667|-113.01667|   1536.2| 5.7| 335.0| 1609.0| -7.8| -9.4|1019.2|   0.5|
|72475023176|2019-01-01T05:52:00|38.41667|-113.01667|   1536.2| 6.2| 762.0| 4828.0| -8.3|-10.6|1019.6|   0.2|
|72475023176|2019-01-01T06:52:00|38.41667|-113.01667|   1536.2| 4.6| 610.0| 3219.0| -8.9|-11.1|1020.5|   0.0|
|72475023176|2019-01-01T07:52:00|38.41667|-113.01667|   1536.2| 6.7|1280.0|16093.0|-10.0|-13.3|1021.2|   0.0|
+-----------+-------------------+--------+----------+---------+----+------+-------+-----+-----+------+------+
only showi

In [45]:
data5 = data4.withColumn("month", f.month(f.to_timestamp('date', 'yyyy-MM-dd')))
data5.show(5)

+-----------+-------------------+--------+----------+---------+----+------+-------+-----+-----+------+------+-----+-----+
|    station|               date|latitude| longitude|elevation|wind|   cig|    vis| temp|  dew|   slp|liquid|liqhr|month|
+-----------+-------------------+--------+----------+---------+----+------+-------+-----+-----+------+------+-----+-----+
|72475023176|2019-01-01T03:52:00|38.41667|-113.01667|   1536.2| 4.1| 396.0| 2012.0| -8.3|-10.6|1019.3|   0.0|  1.0|    1|
|72475023176|2019-01-01T04:52:00|38.41667|-113.01667|   1536.2| 5.7| 335.0| 1609.0| -7.8| -9.4|1019.2|   0.5|  1.0|    1|
|72475023176|2019-01-01T05:52:00|38.41667|-113.01667|   1536.2| 6.2| 762.0| 4828.0| -8.3|-10.6|1019.6|   0.2|  1.0|    1|
|72475023176|2019-01-01T06:52:00|38.41667|-113.01667|   1536.2| 4.6| 610.0| 3219.0| -8.9|-11.1|1020.5|   0.0|  1.0|    1|
|72475023176|2019-01-01T07:52:00|38.41667|-113.01667|   1536.2| 6.7|1280.0|16093.0|-10.0|-13.3|1021.2|   0.0|  1.0|    1|
+-----------+-----------

In [46]:
data5.cache()

DataFrame[station: string, date: string, latitude: float, longitude: float, elevation: float, wind: double, cig: float, vis: float, temp: double, dew: double, slp: double, liquid: double, liqhr: float, month: int]

In [47]:
data5.count()

10667288

In [48]:
data5.printSchema()

root
 |-- station: string (nullable = true)
 |-- date: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- elevation: float (nullable = true)
 |-- wind: double (nullable = true)
 |-- cig: float (nullable = true)
 |-- vis: float (nullable = true)
 |-- temp: double (nullable = true)
 |-- dew: double (nullable = true)
 |-- slp: double (nullable = true)
 |-- liquid: double (nullable = true)
 |-- liqhr: float (nullable = true)
 |-- month: integer (nullable = true)



In [49]:
data5.write.csv("s3://open-data-noaa/cleaneddata", sep=',', header=True,mode="overwrite")

In [50]:
sc.stop()